In [98]:
# Import library
import re
import sys
import os
import shutil
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time
import pandas as pd

In [99]:
def relatedness(model):
    # Baca file rw untuk evaluasi word embedding dengan teknik relatedness cbow
    dir_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataEvaluasiGitHub/word-similarity/monolingual/en/rw.csv'
    list_model_similarity = []
    words_exist = []
    words_not_exist = []
    list_human_score = []
    
    df = pd.read_csv(dir_file)
    list_human_score = []
    
    for index, row in df.iterrows():
        word_1 = row["word1"].lower()
        word_2 = row["word2"].lower()
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model.wv[word_1])
            
            if word_1 not in words_exist:
                words_exist.append(word_1)
            
            wv_2.append(model.wv[word_2])
            
            if word_2 not in words_exist:
                words_exist.append(word_2)
                
            similarity = model.wv.similarity(word_1, word_2)
            list_model_similarity.append(similarity)
            
            human_score = float(row["similarity"])
            list_human_score.append(human_score)
            
        except:
            if word_1 in words_exist: # kata 2 yang tidak ada di model
                if word_2 not in words_not_exist:
                    words_not_exist.append(word_2)
            
            if word_2 in words_exist: # kata 1 yang tidak ada di model
                if word_1 not in words_not_exist:
                    words_not_exist.append(word_1)
        
    
   
        

    rho_model, p_model = spearmanr(list_model_similarity, list_human_score)
    print("banyak kata ada di model", len(words_exist))
    print("banyak kata tidak ada di model", len(words_not_exist))
    return rho_model


In [100]:
# # method untuk evaluasi word analogy
# def word_analogy(model):
#     # Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan cara manual) cbow 1

#     dir_path_word_analogy = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Word_analogy/Word_Analogy.txt'

#     true_predicted = 0
#     false_predicted = 0

#     list_of_words = model.wv.index_to_key



#     with open(dir_path_word_analogy , 'r') as file:
#         content = file.read()
#         lines = content.splitlines() 
#         for line in lines:
#             item_in_line = line.split(" ")
#             word_predicted_curr = ""
#             if len(item_in_line) == 4: # untuk buang baris yang merupakan keterangan
#                 max_value = -sys.maxsize - 1 # diisi dengan min value dahulu
#                 word_1 = item_in_line[0].lower()
#                 word_2 = item_in_line[1].lower()
#                 word_3 = item_in_line[2].lower()
#                 word_4 = item_in_line[3].lower()
#                 try:
#                     word_predicted_curr = model.wv.most_similar(positive=[word_2, word_3], negative=[word_1], topn=1)[0][0]
                    
#                 except Exception as e:
#                     continue
#                 if word_predicted_curr == word_4:
#                     true_predicted += 1
#                 else:
#                     false_predicted += 1

#     # print("true_predicted", true_predicted)
#     # print("false_predicted", false_predicted)
#     print("true_predicted", true_predicted, true_predicted / (true_predicted+false_predicted))
#     print("false_predicted", false_predicted, false_predicted / (true_predicted+false_predicted))


In [101]:
def word_analogy(model):
    # Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan library)

    dir_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataEvaluasiGitHub/word-analogy/monolingual/en/semeval.csv'
    df = pd.read_csv(dir_file)
    true_predicted = 0
    false_predicted = 0
    words_exist = []
    words_not_exist = []
    
    for index, row in df.iterrows():
        word_1 = row["word1"].lower()
        word_2 = row["word2"].lower()
        word_3 = row["word3"].lower()
        word_4 = row["target"].lower()
        word_predicted_curr = ""
        try:
            word_predicted_curr = model.wv.most_similar(positive=[word_2, word_3], negative=[word_1], topn=1)[0][0]
         
            if word_predicted_curr == word_4:
                true_predicted += 1
            else:
                false_predicted += 1
            
            if word_1 not in words_exist:
                words_exist.append(word_1)
            
            if word_2 not in words_exist:
                words_exist.append(word_2)
            
            if word_3 not in words_exist:
                words_exist.append(word_3)
            
            if word_4 not in words_exist:
                words_exist.append(word_4)

        except:
            continue
            
        # untuk menambahkan word_1 jika tidak ada di model    
        try:
            word_vector = model.wv[word_1]
        except:
            if word_1 not in words_not_exist:
                    words_not_exist.append(word_1)
        
        # untuk menambahkan word_2 jika tidak ada di model    
        try:
            word_vector = model.wv[word_2]
        except:
            if word_2 not in words_not_exist:
                    words_not_exist.append(word_2)

        # untuk menambahkan word_3 jika tidak ada di model    
        try:
            word_vector = model.wv[word_3]
        except:
            if word_3 not in words_not_exist:
                    words_not_exist.append(word_3)
        
        # untuk menambahkan word_4 jika tidak ada di model    
        try:
            word_vector = model.wv[word_4]
        except:
            if word_4 not in words_not_exist:
                    words_not_exist.append(word_4)

    print("true_predicted", true_predicted, true_predicted / (true_predicted+false_predicted))
    print("false_predicted", false_predicted, false_predicted / (true_predicted+false_predicted))
    print("banyak kata ada di model", len(words_exist))
    print("banyak kata tidak ada di model", len(words_not_exist))

In [102]:
# Ambil file asli dari folder
directory_path_real = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_asli'

files_real = os.listdir(directory_path_real)
list_files_real = []

# Ambil daftar nama file
for file in files_real:
    if '.DS_Store' not in file:
        list_files_real.append(file)
        
all_text_real = ""
for file in list_files_real:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_asli/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_real += content


In [103]:
# Ambil file parafrasa dari folder
directory_path_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa'

files_paraphrased = os.listdir(directory_path_paraphrased)
list_files_paraphrased = []

# Ambil daftar nama file
for file in files_paraphrased:
    if '.DS_Store' not in file:
        list_files_paraphrased.append(file)
        
all_text_paraphrased = ""
for file in list_files_paraphrased:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_paraphrased += content
        


In [104]:
# Melakukan lowercase lagi
all_text_real = all_text_real.lower()
all_text_paraphrased = all_text_paraphrased.lower()

In [105]:
# Memisahkan text per kalimat (menggunakan delimiter titik)
list_of_sentences_real = []
list_of_sentences_paraphrased = []

list_of_sentences_real = all_text_real.split('.')
list_of_sentences_paraphrased = all_text_paraphrased.split('.')

print(list_of_sentences_real[0:10])
print(list_of_sentences_paraphrased[0:10])

['computer-aided structure-activity based prediction studies in drug design helps to treat diseases with novel biomarkers', ' prediction of activity spectra for substances(pass) database comprised 46,000 biologically well-known active drugs and screening are performed before the establishment of an in vitro experiment', ' pass gives the significant bioactivities of chemical compounds as pa(probable activity) and pi(probable inactivity) values to mention the compounds, whether they are active are inactive', ' the pa values higher than 0', '7 indicated this compound would be active in experiment and pi values indicate theirs inactivate possibilities', ' the admetsar chemoinformatics based tool used to predict absorption, metabolism, excretion, and toxicity of the particular compound', ' based on these criteria, the outcomes of an in vitro experiment will lower the risk of negative results[13,14]', '\n\n\n\nclc-pred tools performed to predict cytotoxicity of tumor cell lines, and it is ba

In [106]:
# melakukan tokenisasi dan menggabungkan teks asli dan parafrasa


list_of_sentences_for_model = []

tokenizer = RegexpTokenizer(r'[a-zA-Z]+')

for sentence in list_of_sentences_real:
    sentence_tokenized = tokenizer.tokenize(sentence)
    list_of_sentences_for_model.append(sentence_tokenized)

for sentence in list_of_sentences_paraphrased:
    sentence_tokenized = tokenizer.tokenize(sentence)
    list_of_sentences_for_model.append(sentence_tokenized)

print(list_of_sentences_for_model[0:10])


[['computer', 'aided', 'structure', 'activity', 'based', 'prediction', 'studies', 'in', 'drug', 'design', 'helps', 'to', 'treat', 'diseases', 'with', 'novel', 'biomarkers'], ['prediction', 'of', 'activity', 'spectra', 'for', 'substances', 'pass', 'database', 'comprised', 'biologically', 'well', 'known', 'active', 'drugs', 'and', 'screening', 'are', 'performed', 'before', 'the', 'establishment', 'of', 'an', 'in', 'vitro', 'experiment'], ['pass', 'gives', 'the', 'significant', 'bioactivities', 'of', 'chemical', 'compounds', 'as', 'pa', 'probable', 'activity', 'and', 'pi', 'probable', 'inactivity', 'values', 'to', 'mention', 'the', 'compounds', 'whether', 'they', 'are', 'active', 'are', 'inactive'], ['the', 'pa', 'values', 'higher', 'than'], ['indicated', 'this', 'compound', 'would', 'be', 'active', 'in', 'experiment', 'and', 'pi', 'values', 'indicate', 'theirs', 'inactivate', 'possibilities'], ['the', 'admetsar', 'chemoinformatics', 'based', 'tool', 'used', 'to', 'predict', 'absorption',

In [107]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 1

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=20, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.01, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 0 minutes 58 seconds


In [108]:
# save model word embedding dengan teknik word2vec dengan cbow 1
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_1.model")


In [109]:
# load model word embedding dengan teknik word2vec dengan cbow 1
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_1.model")

In [110]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 1
training_loss = model.get_latest_training_loss()
print(training_loss)

14459750.0


In [111]:
# nilai relatedness model cbow 1
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
-0.014615062367807321


In [112]:
# nilai word analogy model cbow 1
word_analogy(model)

true_predicted 5 0.0026041666666666665
false_predicted 1915 0.9973958333333334
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [113]:
# Membuat model word embedding dengan teknik word2vec dengan sg 1

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=20, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.01, seed = 10, epochs = 100)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 2 minutes 51 seconds


In [114]:
# save model word embedding dengan teknik word2vec dengan sg 1
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_1.model")


In [115]:
# load model word embedding dengan teknik word2vec dengan sg 1
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_1.model")


In [116]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 1
training_loss = model.get_latest_training_loss()
print(training_loss)

45895132.0


In [117]:
# nilai relatedness model sg 1
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.21749188332567632


In [118]:
# nilai word analogy model sg 1
word_analogy(model)

true_predicted 6 0.003125
false_predicted 1914 0.996875
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [119]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 2

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=20, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.001, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 0 minutes 28 seconds


In [120]:
# save model word embedding dengan teknik word2vec dengan cbow 2
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_2.model")


In [121]:
# load model word embedding dengan teknik word2vec dengan cbow 2
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_2.model")


In [122]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 2
training_loss = model.get_latest_training_loss()
print(training_loss)


16990444.0


In [123]:
# nilai relatedness model cbow 2
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
-0.03805784519922117


In [124]:
# nilai word analogy model cbow 2
word_analogy(model)


true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [125]:
# Membuat model word embedding dengan teknik word2vec dengan sg 2

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=20, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.001, seed = 10, epochs = 100)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 1 minutes 18 seconds


In [126]:
# save model word embedding dengan teknik word2vec dengan sg 2
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_2.model")


In [127]:
# load model word embedding dengan teknik word2vec dengan sg 2
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_2.model")


In [128]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 2
training_loss = model.get_latest_training_loss()
print(training_loss)

49390352.0


In [129]:
# nilai relatedness model sg 2
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.18296383373385983


In [130]:
# nilai word analogy model sg 2
word_analogy(model)

true_predicted 7 0.0036458333333333334
false_predicted 1913 0.9963541666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [131]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 3

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=20, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.1, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 1 minutes 0 seconds


In [132]:
# save model word embedding dengan teknik word2vec dengan cbow 3
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_3.model")


In [133]:
# load model word embedding dengan teknik word2vec dengan cbow 3
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_3.model")


In [134]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 3
training_loss = model.get_latest_training_loss()
print(training_loss)


13958812.0


In [135]:
# nilai relatedness model cbow 3
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
-0.02875774621629753


In [136]:
# nilai word analogy model cbow 3
word_analogy(model)


true_predicted 3 0.0015625
false_predicted 1917 0.9984375
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [137]:
# Membuat model word embedding dengan teknik word2vec dengan sg 3

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=20, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.1, seed = 10, epochs = 100)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 2 minutes 51 seconds


In [138]:
# save model word embedding dengan teknik word2vec dengan sg 3
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_3.model")


In [139]:
# load model word embedding dengan teknik word2vec dengan sg 3
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_3.model")


In [140]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 3
training_loss = model.get_latest_training_loss()
print(training_loss)

45273248.0


In [141]:
# nilai relatedness model sg 3
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.14590797152822502


In [142]:
# nilai word analogy model sg 3
word_analogy(model)

true_predicted 3 0.0015625
false_predicted 1917 0.9984375
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [143]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 4

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.01, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 0 minutes 53 seconds


In [144]:
# save model word embedding dengan teknik word2vec dengan cbow 4
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_4.model")


In [145]:
# load model word embedding dengan teknik word2vec dengan cbow 4
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_4.model")


In [146]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 4
training_loss = model.get_latest_training_loss()
print(training_loss)


15350427.0


In [147]:
# nilai relatedness model cbow 4
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
-0.0027476683873612164


In [148]:
# nilai word analogy model cbow 4
word_analogy(model)


true_predicted 3 0.0015625
false_predicted 1917 0.9984375
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [149]:
# Membuat model word embedding dengan teknik word2vec dengan sg 4

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.01, seed = 10, epochs = 100)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 3 minutes 3 seconds


In [150]:
# save model word embedding dengan teknik word2vec dengan sg 4
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_4.model")


In [151]:
# load model word embedding dengan teknik word2vec dengan sg 4
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_4.model")


In [152]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 4
training_loss = model.get_latest_training_loss()
print(training_loss)

46725216.0


In [153]:
# nilai relatedness model sg 4
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.148249300383298


In [154]:
# nilai word analogy model sg 4
word_analogy(model)

true_predicted 0 0.0
false_predicted 1920 1.0
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [155]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 5

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.001, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 1 minutes 9 seconds


In [156]:
# save model word embedding dengan teknik word2vec dengan cbow 5
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_5.model")


In [157]:
# load model word embedding dengan teknik word2vec dengan cbow 5
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_5.model")


In [158]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 5
training_loss = model.get_latest_training_loss()
print(training_loss)


17342380.0


In [159]:
# nilai relatedness model cbow 5
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
-0.006775694074841656


In [160]:
# nilai word analogy model cbow 5
word_analogy(model)


true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [161]:
# Membuat model word embedding dengan teknik word2vec dengan sg 5

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.001, seed = 10, epochs = 100)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 2 minutes 20 seconds


In [162]:
# save model word embedding dengan teknik word2vec dengan sg 5
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_5.model")


In [163]:
# load model word embedding dengan teknik word2vec dengan sg 5
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_5.model")


In [164]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 5
training_loss = model.get_latest_training_loss()
print(training_loss)

49249468.0


In [165]:
# nilai relatedness model sg 5
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.18637999998102084


In [166]:
# nilai word analogy model sg 5
word_analogy(model)

true_predicted 2 0.0010416666666666667
false_predicted 1918 0.9989583333333333
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [167]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 6

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.1, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 1 minutes 7 seconds


In [168]:
# save model word embedding dengan teknik word2vec dengan cbow 6
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_6.model")


In [169]:
# load model word embedding dengan teknik word2vec dengan cbow 6
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_6.model")


In [170]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 6
training_loss = model.get_latest_training_loss()
print(training_loss)


15334991.0


In [171]:
# nilai relatedness model cbow 6
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
0.02209004116117041


In [172]:
# nilai word analogy model cbow 6
word_analogy(model)


true_predicted 3 0.0015625
false_predicted 1917 0.9984375
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [173]:
# Membuat model word embedding dengan teknik word2vec dengan sg 6

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.1, seed = 10, epochs = 100)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 1 minutes 9 seconds


In [174]:
# save model word embedding dengan teknik word2vec dengan sg 6
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_6.model")


In [175]:
# load model word embedding dengan teknik word2vec dengan sg 6
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_6.model")


In [176]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 6
training_loss = model.get_latest_training_loss()
print(training_loss)

45994752.0


In [177]:
# nilai relatedness model sg 6
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.1271137693495935


In [178]:
# nilai word analogy model sg 6
word_analogy(model)

true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [179]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 7

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.1, seed = 10, epochs = 1000)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 9 minutes 11 seconds


In [180]:
# save model word embedding dengan teknik word2vec dengan cbow 7
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_7.model")


In [181]:
# load model word embedding dengan teknik word2vec dengan cbow 7
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_7.model")


In [182]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 7
training_loss = model.get_latest_training_loss()
print(training_loss)


76664208.0


In [183]:
# nilai relatedness model cbow 7
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
-0.002951425688977106


In [184]:
# nilai word analogy model cbow 7
word_analogy(model)


true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [185]:
# Membuat model word embedding dengan teknik word2vec dengan sg 7

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.1, seed = 10, epochs = 1000)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 22 minutes 26 seconds


In [186]:
# save model word embedding dengan teknik word2vec dengan sg 7
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_7.model")


In [187]:
# load model word embedding dengan teknik word2vec dengan sg 7
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_7.model")


In [188]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 7
training_loss = model.get_latest_training_loss()
print(training_loss)

81604304.0


In [189]:
# nilai relatedness model sg 7
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.15355521572191616


In [190]:
# nilai word analogy model sg 7
word_analogy(model)

true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [191]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 8

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.1, seed = 10, epochs = 5000)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 40 minutes 40 seconds


In [192]:
# save model word embedding dengan teknik word2vec dengan cbow 8
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_8.model")


In [193]:
# load model word embedding dengan teknik word2vec dengan cbow 8
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_8.model")


In [194]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 8
training_loss = model.get_latest_training_loss()
print(training_loss)


134217728.0


In [195]:
# nilai relatedness model cbow 8
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
0.0034072357079898146


In [196]:
# nilai word analogy model cbow 8
word_analogy(model)


true_predicted 2 0.0010416666666666667
false_predicted 1918 0.9989583333333333
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [197]:
# Membuat model word embedding dengan teknik word2vec dengan sg 8

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.1, seed = 10, epochs = 5000)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 1 hours 17 minutes 31 seconds


In [198]:
# save model word embedding dengan teknik word2vec dengan sg 8
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_8.model")


In [199]:
# load model word embedding dengan teknik word2vec dengan sg 8
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_8.model")


In [200]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 8
training_loss = model.get_latest_training_loss()
print(training_loss)

134217728.0


In [201]:
# nilai relatedness model sg 8
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.15468069867167222


In [202]:
# nilai word analogy model sg 8
word_analogy(model)

true_predicted 2 0.0010416666666666667
false_predicted 1918 0.9989583333333333
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [203]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 9

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.01, seed = 10, epochs = 5000)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 41 minutes 27 seconds


In [204]:
# save model word embedding dengan teknik word2vec dengan cbow 9
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_9.model")


In [205]:
# load model word embedding dengan teknik word2vec dengan cbow 9
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_9.model")


In [206]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 9
training_loss = model.get_latest_training_loss()
print(training_loss)


134217728.0


In [207]:
# nilai relatedness model cbow 9
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
0.02974762597916504


In [208]:
# nilai word analogy model cbow 9
word_analogy(model)


true_predicted 2 0.0010416666666666667
false_predicted 1918 0.9989583333333333
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [209]:
# Membuat model word embedding dengan teknik word2vec dengan sg 9

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 1, alpha = 0.01, seed = 10, epochs = 5000)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 2 hours 14 minutes 20 seconds


In [210]:
# save model word embedding dengan teknik word2vec dengan sg 9
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_9.model")


In [211]:
# load model word embedding dengan teknik word2vec dengan sg 9
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_9.model")


In [212]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 9
training_loss = model.get_latest_training_loss()
print(training_loss)

121615504.0


In [213]:
# nilai relatedness model sg 9
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.16645690714689693


In [214]:
# nilai word analogy model sg 9
word_analogy(model)

true_predicted 2 0.0010416666666666667
false_predicted 1918 0.9989583333333333
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [215]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 10

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=7, sg = 0, cbow_mean = 0, alpha = 0.01, seed = 10, epochs = 5000)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 52 minutes 36 seconds


In [216]:
# save model word embedding dengan teknik word2vec dengan cbow 10
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_10.model")


In [217]:
# load model word embedding dengan teknik word2vec dengan cbow 10
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_10.model")


In [218]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 10
training_loss = model.get_latest_training_loss()
print(training_loss)


134217728.0


In [219]:
# nilai relatedness model cbow 10
print(relatedness(model))


banyak kata ada di model 646
banyak kata tidak ada di model 255
0.021115084799286303


In [220]:
# nilai word analogy model cbow 10
word_analogy(model)


true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [221]:
# Membuat model word embedding dengan teknik word2vec dengan sg 10

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=7, sg = 1, alpha = 0.01, seed = 10, epochs = 5000)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 2 hours 54 minutes 28 seconds


In [222]:
# save model word embedding dengan teknik word2vec dengan sg 10
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_10.model")


In [223]:
# load model word embedding dengan teknik word2vec dengan sg 10
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_10.model")


In [224]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 10
training_loss = model.get_latest_training_loss()
print(training_loss)

129177072.0


In [225]:
# nilai relatedness model sg 10
print(relatedness(model))

banyak kata ada di model 646
banyak kata tidak ada di model 255
0.1549939725852293


In [226]:
# nilai word analogy model sg 10
word_analogy(model)

true_predicted 1 0.0005208333333333333
false_predicted 1919 0.9994791666666667
banyak kata ada di model 1544
banyak kata tidak ada di model 356


In [ ]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 11

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=9, sg = 0, cbow_mean = 0, alpha = 0.01, seed = 10, epochs = 5000)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



In [ ]:
# save model word embedding dengan teknik word2vec dengan cbow 11
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_11.model")


In [ ]:
# load model word embedding dengan teknik word2vec dengan cbow 11
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_11.model")


In [ ]:
# Melihat training loss model word embedding dengan teknik word2vec dengan cbow 11
training_loss = model.get_latest_training_loss()
print(training_loss)


In [ ]:
# nilai relatedness model cbow 11
print(relatedness(model))


In [ ]:
# nilai word analogy model cbow 11
word_analogy(model)


In [ ]:
# Membuat model word embedding dengan teknik word2vec dengan sg 11

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=9, sg = 1, alpha = 0.01, seed = 10, epochs = 5000)


# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



In [ ]:
# save model word embedding dengan teknik word2vec dengan sg 11
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_11.model")


In [ ]:
# load model word embedding dengan teknik word2vec dengan sg 11
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_sg_11.model")


In [ ]:
# Melihat training loss model word embedding dengan teknik word2vec dengan sg 11
training_loss = model.get_latest_training_loss()
print(training_loss)

In [ ]:
# nilai relatedness model sg 11
print(relatedness(model))

In [ ]:
# nilai word analogy model sg 11
word_analogy(model)